# **BUSCADOR DE HIPERPARAMETROS DE ARF Y HT**

**IMPORTACIÓN DE LIBERIAS**

In [1]:
!pip install river==0.21.2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import json, math, pandas as pd
from itertools import product
from river import metrics, tree
from river.forest import ARFClassifier  # River 0.21.x


**CARGA Y LIMPIEZA DEL DATASET**

In [26]:
CSV_PATH = "/content/drive/MyDrive/Colab Notebooks/TESIS/FASE ONLINE/ARCHIVOS ONLINE/windows_hiperparametros.csv"

BAD_LABELS = {"", " ", "null", "NULL", "None", None}

def load_windows(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path, parse_dates=["start_time","end_time"], low_memory=False)

    def _norm_lbl(v):
        if pd.isna(v): return None
        s = str(v).strip()
        return s if s not in BAD_LABELS else None

    df["etiqueta"] = df["etiqueta"].map(_norm_lbl)
    df = df[df["etiqueta"].notna()].copy()

    for col in ["id_usuario", "session_id"]:
        if col not in df.columns:
            df[col] = 0
    df = df.sort_values(["id_usuario","session_id","start_time","id"], kind="mergesort").reset_index(drop=True)

    if "features" not in df.columns:
        raise ValueError("El CSV debe tener la columna 'features' con JSON de features")
    return df

**FUNCIÓN PARA DESCUBRIR CARACTERÍSTICAS Y FORMAR EL VECTOR X**

In [ ]:
def discover_feature_names(df):
    names = set()
    for v in df["features"].head(500):
        try:
            d = v if isinstance(v, dict) else json.loads(v)
            names.update(d.keys())
        except Exception:
            pass
    if not names:
        names = set(json.loads(df["features"].iloc[0]).keys())
    return sorted(names)

def row_to_x(row, feat_names):
    d = row["features"]
    d = d if isinstance(d, dict) else json.loads(d)
    return {f: float(d.get(f, 0.0)) for f in feat_names}

**FUNCIÓN DE EVALUACIÓN PRECUENCIAL (Predict --> Update Metrics --> Learn)**

In [ ]:
def eval_prequential(df, ht_params: dict, arf_params: dict | None, feat_names: list[str]):
    # HT
    A = tree.HoeffdingTreeClassifier(**ht_params)
    mA = metrics.Accuracy()

    # ARF (None = OFF). OJO: en River 0.21 el parámetro de semilla es 'seed'
    if arf_params is not None:
        # No metas random_state. Si quieres reproducible, agrega 'seed': 42 en el grid o aquí.
        B = ARFClassifier(**arf_params)
        mB = metrics.Accuracy()
    else:
        B = None; mB = None

    for _, r in df.iterrows():
        x = row_to_x(r, feat_names)
        y = r["etiqueta"]

        yhatA = A.predict_one(x)
        if yhatA is not None:
            mA.update(y, yhatA)
        A.learn_one(x, y)

        if B is not None:
            yhatB = B.predict_one(x)
            if yhatB is not None:
                mB.update(y, yhatB)
            B.learn_one(x, y)

    return float(mA.get()), (float(mB.get()) if mB else None)

**FUNCIÓN DE BUSQUEDA DE HIPERPARAMETROS (GridSearch)**

In [ ]:
def grid_search(df, ht_grid: dict, arf_grid: dict | None):
    feat_names = discover_feature_names(df)
    best = {"A_acc": -math.inf, "B_acc": -math.inf, "ht": None, "arf": None}

    # --- HT combos ---
    ht_items = sorted(ht_grid.items())
    ht_keys  = [k for k, _ in ht_items]
    ht_vals  = [v for _, v in ht_items]

    # --- ARF combos ---
    arf_enabled = (arf_grid is not None)
    if arf_enabled:
        arf_items = sorted(arf_grid.items())
        if arf_items:
            arf_keys = [k for k, _ in arf_items]
            arf_vals = [v for _, v in arf_items]
            arf_iter = product(*arf_vals)
        else:
            # {} → una corrida con defaults
            arf_keys = []
            arf_iter = [()]
    else:
        arf_keys = []
        arf_iter = [None]

    valid_arf_runs = 0
    skipped_arf_runs = 0

    for ht_tuple in product(*ht_vals):
        ht_params = dict(zip(ht_keys, ht_tuple))

        for arf_tuple in arf_iter:
            if arf_tuple is None:
                # ARF OFF
                A_acc, B_acc = eval_prequential(df, ht_params, None, feat_names)
            else:
                # Construye params de ARF y prueba; si el combo no lo soporta la versión, se salta
                arf_params = dict(zip(arf_keys, arf_tuple)) if arf_keys else {}
                try:
                    A_acc, B_acc = eval_prequential(df, ht_params, arf_params, feat_names)
                    valid_arf_runs += 1
                    # Guarda mejores para B
                    if B_acc is not None and B_acc > best["B_acc"]:
                        best["B_acc"], best["arf"] = B_acc, arf_params.copy()
                except (TypeError, ValueError) as e:
                    skipped_arf_runs += 1
                    continue

            # Guarda mejores para A
            if A_acc > best["A_acc"]:
                best["A_acc"], best["ht"] = A_acc, ht_params.copy()

    return best, valid_arf_runs, skipped_arf_runs

# ===== RUN =====
df = load_windows(CSV_PATH)
df[["id","id_usuario","session_id","start_time","end_time","etiqueta","features"]].to_csv("windows_clean.csv", index=False)

**OPCIONES DE HIPERPARAMETROS**

In [25]:
# Grids (HT y ARF).
ht_grid = {
    "grace_period": [30, 50],
    "delta": [1e-5, 1e-6],
    "leaf_prediction": ["mc", "nb"],
    # "max_depth": [None, 20], "split_criterion": ["gini","info_gain"]
}

arf_grid = {
    "n_models": [8, 12, 16],
    "max_features": ["sqrt", "log2"],
    "lambda_value": [4, 6, 8],
    # "grace_period": [30, 50],
    # "split_criterion": ["gini", "info_gain"],
    # "leaf_prediction": ["mc", "nb"],
    # Para reproducible:
    "seed": [42],
}

best, n_ok, n_skipped = grid_search(df, ht_grid, arf_grid)

print("\n=== MEJORES HIPERPARÁMETROS (global) ===")
print(f"HT  (A): {best['ht']} → acc: {round(best['A_acc'], 4)}")
print(f"ARF (B): {best['arf']} → acc: {round(best['B_acc'], 4) if best['arf'] is not None else None}")
print(f"(ARF combos válidos: {n_ok}, combos saltados por incompatibilidad: {n_skipped})")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

=== MEJORES HIPERPARÁMETROS (global) ===
HT  (A): {'delta': 1e-05, 'grace_period': 30, 'leaf_prediction': 'mc'} → acc: 0.3867
ARF (B): {'lambda_value': 8, 'max_features': 'sqrt', 'n_models': 12, 'seed': 42} → acc: 0.8608
(ARF combos válidos: 18, combos saltados por incompatibilidad: 0)
